# LLM Detect: Text Cluster [中文]
[Link](https://www.kaggle.com/code/finlay/llm-detect-text-cluster)

In [1]:
!pip install sentence_transformers > /dev/null

�t�Χ䤣����w�����|�C


## Read data

In [2]:
import pandas as pd

# 数据路径
#DATA_PATH = './input'
#DATA_PATH = '/kaggle/input/llm-detect-ai-generated-text'
DATA_PATH = 'D:/NTUT-CSIE-Master-Course/113-1/NTHU-Natural-Language-Processing/Term_Project/input_data'

# 读取训练集文章数据
train_essays = pd.read_csv(f'{DATA_PATH}/train_essays.csv')

# 读取训练集作文题目数据
train_prompts = pd.read_csv(f'{DATA_PATH}/train_prompts.csv')

# 读取测试集文章数据
test_essays = pd.read_csv(f'{DATA_PATH}/test_essays.csv')

# 读取样本提交文件
sample_submit = pd.read_csv(f'{DATA_PATH}/sample_submission.csv')

In [ ]:
!pip install kaggle
!pip install kagglehub

In [ ]:
import kagglehub
import shutil
path = kagglehub.dataset_download("thedrcat/daigt-v2-train-dataset")
# Move the downloaded dataset to the DATA_PATH
shutil.move(path, DATA_PATH)

  7%|▋         | 2.00M/28.5M [00:00<00:11, 2.41MB/s]

In [ ]:
# 加载外数据集
#train_v2_drcat_02_path = '/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv'
train_v2_drcat_02_path = 'D:/NTUT-CSIE-Master-Course/113-1/NTHU-Natural-Language-Processing/Term_Project/input_data/train_v2_drcat_02.csv'
train = pd.read_csv(train_v2_drcat_02_path, sep=',')

In [ ]:
# 合并外部数据集
# train = pd.concat([
#     train_essays.rename({'generated': 'label'}, axis=1)[['text', 'label']],
#     train[['text', 'label']]
# ],axis=0)

# 按照文本内容进行去重
train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)

## Encode Text

In [ ]:
!pip install -U sentence-transformers
!pip install tf-keras

In [ ]:
from sentence_transformers import SentenceTransformer
# 加载与训练模型
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# 进行文本编码
embeddings = model.encode(train['text'], show_progress_bar=True)

## PCA

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings)

In [ ]:
import matplotlib.pyplot as plt

for c in train['label'].unique():
    plt.scatter(embeddings_pca[train['label']==c, 0],
                embeddings_pca[train['label']==c, 1])
plt.legend(train['label'].unique(), title='label')

In [ ]:
for c in train['prompt_name'].unique():
    plt.scatter(embeddings_pca[train['prompt_name']==c, 0],
                embeddings_pca[train['prompt_name']==c, 1])
plt.legend(train['prompt_name'].unique(), title='prompt_name', bbox_to_anchor=(1, 1.0))

## TSNE

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
X = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
X_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(X)
X_embedded.shape

In [ ]:
data_index = np.random.choice(range(44868), 5000)

tsne = TSNE(n_components=2)
embeddings_tsne = tsne.fit_transform(embeddings[data_index])

In [ ]:
for c in train['label'].iloc[data_index].unique():
    plt.scatter(embeddings_tsne[train['label'].iloc[data_index]==c, 0],
                embeddings_tsne[train['label'].iloc[data_index]==c, 1])

plt.legend(train['label'].iloc[data_index].unique(), title='label')

In [ ]:
for c in train['prompt_name'].iloc[data_index].unique():
    plt.scatter(embeddings_tsne[train['prompt_name'].iloc[data_index]==c, 0],
                embeddings_tsne[train['prompt_name'].iloc[data_index]==c, 1])

plt.legend(train['prompt_name'].unique(), title='prompt_name', bbox_to_anchor=(1, 1.0))